In [6]:
"""
KPI-Report MVP (analyst ▸ writer), Autogen 0.2.x
"""

from __future__ import annotations
import os
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group.patterns import AutoPattern

# ──────────────────────────────────────────────────────────────────────
# 1. LLM config  (uses your METIS wrapper)
# ──────────────────────────────────────────────────────────────────────
OPENAI_API_KEY = "tpsg-AYCymq4XvkEmdyoL1Na0cujGfY0aeqys"


MODEL_NAME = "gpt-4o-mini"

config_list = [
    {
        "model": MODEL_NAME,
        "api_key": "tpsg-AYCymq4XvkEmdyoL1Na0cujGfY0aeqy",
        "base_url": "https://api.metisai.ir/api/v1/wrapper/openai_chat_completion"
    }
]

llm_config = LLMConfig(config_list=config_list, temperature=1)

# ──────────────────────────────────────────────────────────────────────
# 2. Role prompts (merged + feedback)
# ──────────────────────────────────────────────────────────────────────
ANALYST_WRITER_SYS_MSG = """
You are ANALYST_WRITER.

Step 1 – Analysis
• Input: ≤ 5 KPI JSON blobs + plain-text targets (e.g.  impressions = 150000).
• Parse mentally, no code.
• Typical derived metrics:
    conversion_rate = conversions / impressions × 100
    ctr             = clicks / impressions × 100
• Compare each KPI to its target and create a bullet list:

    KPI_NAME
        current = …
        target  = …
        delta   = +X or –Y
        comment = …

Step 2 – Report  
Produce Markdown ending **REPORT_DRAFT**:

## Executive Summary
<~2 sentences>

## KPI Details
- KPI A – …

## Recommendations
- ≥ 2 bullets if any KPI below target, otherwise “All KPIs meet or exceed target.”
"""

FEEDBACK_SYS_MSG_TEMPLATE = """
You are the FEEDBACK agent.

The user provided this input:

{user_input}

You will now read a report based on that input and check:
- Are all KPIs mentioned and correctly calculated?
- Is there any mismatch with the target values?
- Is the writing clear and actionable?

Suggest improvements in a few bullet points.
If everything looks great, say so.

End your reply with **FEEDBACK_DONE**.
"""

def create_analyst_writer():
    return ConversableAgent(
        name="analyst_writer",
        system_message=ANALYST_WRITER_SYS_MSG,
        llm_config=llm_config,
    )

def create_feedback_agent(user_input: str):
    return ConversableAgent(
        name="feedback",
        system_message=FEEDBACK_SYS_MSG_TEMPLATE.format(user_input=user_input),
        llm_config=llm_config,
    )

def create_user():
    return ConversableAgent(
        name="user",
        system_message="You provide KPI JSON and targets, then wait for the report.",
        llm_config=False,
        human_input_mode="ALWAYS",
    )


# ──────────────────────────────────────────────────────────────────────
# 5. Helper: JSON list + targets (plain-text) → final report
# ──────────────────────────────────────────────────────────────────────
def generate_report(json_blobs: list[str], targets: str = "") -> str:
    payload = "\n\n".join(f"```json\n{blob}\n```" for blob in json_blobs)

    # Full user input text
    user_input = f"Here are the KPI blobs:\n{payload}\n\nTargets:\n{targets}"

    # Create agents dynamically
    analyst_writer = create_analyst_writer()
    feedback = create_feedback_agent(user_input)
    user = create_user()

    # Create conversation pattern
    pattern = AutoPattern(
        initial_agent=analyst_writer,
        agents=[analyst_writer, feedback],
        user_agent=user,
        group_manager_args={"llm_config": llm_config},
    )

    chat, *_ = initiate_group_chat(pattern=pattern, messages=user_input, max_rounds=10)
    return chat.chat_history[-1]["content"]

# ──────────────────────────────────────────────────────────────────────
# 6. Demo run
# ──────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    JSON_IMPRESSIONS = """
    {
    "name": "impressions",
    "data": [
        {"date": "2025-07-01", "value": 108200},
        {"date": "2025-07-08", "value": 114500},
        {"date": "2025-07-15", "value": 121300},
        {"date": "2025-07-22", "value": 132800},
        {"date": "2025-07-29", "value": 127600},
        {"date": "2025-08-05", "value": 139400},
        {"date": "2025-08-12", "value": 148150},
        {"date": "2025-08-19", "value": 156700}
        ]
    }

    """

    JSON_CONVERSIONS = """
    {
    "name": "conversions",
    "data": [
        {"date": "2025-07-01", "value": 1040},
        {"date": "2025-07-08", "value": 1105},
        {"date": "2025-07-15", "value": 1170},
        {"date": "2025-07-22", "value": 1260},
        {"date": "2025-07-29", "value": 1195},
        {"date": "2025-08-05", "value": 1340},
        {"date": "2025-08-12", "value": 1415},
        {"date": "2025-08-19", "value": 1510}
    ]
    }
    """

    JSON_CLICKS = """
    {
    "name": "clicks",
    "data": [
        {"date": "2025-07-01", "value": 5300},
        {"date": "2025-07-08", "value": 5600},
        {"date": "2025-07-15", "value": 5900},
        {"date": "2025-07-22", "value": 6400},
        {"date": "2025-07-29", "value": 6100},
        {"date": "2025-08-05", "value": 6800},
        {"date": "2025-08-12", "value": 7250},
        {"date": "2025-08-19", "value": 7700}
    ]
    }
    """

    JSON_AD_SPEND = """
    {
    "name": "ad_spend_usd",
    "data": [
        {"date": "2025-07-01", "value": 28200},
        {"date": "2025-07-08", "value": 29900},
        {"date": "2025-07-15", "value": 31800},
        {"date": "2025-07-22", "value": 34750},
        {"date": "2025-07-29", "value": 33100},
        {"date": "2025-08-05", "value": 36050},
        {"date": "2025-08-12", "value": 38200},
        {"date": "2025-08-19", "value": 40100}
    ]
    }
    """

    JSON_REVENUE = """
    {
    "name": "revenue_usd",
    "data": [
        {"date": "2025-07-01", "value": 94500},
        {"date": "2025-07-08", "value": 99700},
        {"date": "2025-07-15", "value": 105800},
        {"date": "2025-07-22", "value": 113400},
        {"date": "2025-07-29", "value": 109600},
        {"date": "2025-08-05", "value": 118900},
        {"date": "2025-08-12", "value": 125750},
        {"date": "2025-08-19", "value": 133600}
    ]
    }
    """

    TARGETS = """
    impressions = 150000
    conversions = 1500
    revenue_usd = 125000
    """

    report_md = generate_report(
        json_blobs=[
            JSON_IMPRESSIONS,
            JSON_CONVERSIONS,
            JSON_CLICKS,
            JSON_AD_SPEND,
            JSON_REVENUE,
        ],
        targets=TARGETS,
    )

    print(report_md)  # ends with **REPORT_DONE**

user (to chat_manager):

Here are the KPI blobs:
```json

    {
    "name": "impressions",
    "data": [
        {"date": "2025-07-01", "value": 108200},
        {"date": "2025-07-08", "value": 114500},
        {"date": "2025-07-15", "value": 121300},
        {"date": "2025-07-22", "value": 132800},
        {"date": "2025-07-29", "value": 127600},
        {"date": "2025-08-05", "value": 139400},
        {"date": "2025-08-12", "value": 148150},
        {"date": "2025-08-19", "value": 156700}
        ]
    }

    
```

```json

    {
    "name": "conversions",
    "data": [
        {"date": "2025-07-01", "value": 1040},
        {"date": "2025-07-08", "value": 1105},
        {"date": "2025-07-15", "value": 1170},
        {"date": "2025-07-22", "value": 1260},
        {"date": "2025-07-29", "value": 1195},
        {"date": "2025-08-05", "value": 1340},
        {"date": "2025-08-12", "value": 1415},
        {"date": "2025-08-19", "value": 1510}
    ]
    }
    
```

```json

    {
    "name